<a href="https://colab.research.google.com/github/ksknysn/Coursera-https-deeplearning-ai-/blob/main/C2_Device-based-TF-lite/W1/ungraded_labs/my_Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

import pathlib 
import matplotlib.pyplot as plt
from google.colab import files

%matplotlib inline

Create a basic model of Fashion Mnist Data

In [2]:
fmnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


4431872/4422102 [==============================] - 0s 0us/step


In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4753 - accuracy: 0.8292
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3592 - accuracy: 0.8695
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3225 - accuracy: 0.8805
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3005 - accuracy: 0.8899
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2815 - accuracy: 0.8959
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2653 - accuracy: 0.9012
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2531 - accuracy: 0.9059
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2435 - accuracy: 0.9104
Epoch 9/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2317 - accuracy: 0.9136
Epoch 10/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.222

Generate a saved model

In [4]:
export_dir = 'saved_model/2'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/2/assets


Convert the saved model to TFLite

In [5]:
#Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [6]:
tflite_model_file = pathlib.Path('/content/model.tflite')
tflite_model_file.write_bytes(tflite_model)

1629960

Initialize the TFLite interpreter to try it out

In [7]:
#Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

In [8]:
#Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [13]:
input_details

[{'dtype': numpy.float32,
  'index': 0,
  'name': 'serving_default_flatten_input:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([ 1, 28, 28], dtype=int32),
  'shape_signature': array([-1, 28, 28], dtype=int32),
  'sparsity_parameters': {}}]

In [15]:
#Test the TensorFlow Lite model on random input data
input_shape = input_details[0]['shape']
inputs, outputs = [], []
for _ in range(100):
  input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
  interpreter.set_tensor(input_details[0]['index'], input_data)

  interpreter.invoke()
  tflite_results = interpreter.get_tensor(output_details[0]['index'])

  #Test the TensorFlow model on random input data
  tf_results = model(tf.constant(input_data))
  output_data = np.array(tf_results)

  inputs.append(input_data[0][0])
  outputs.append(output_data[0][0])